In [1]:
import pymongo
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')

In [2]:
#This will create a Collection/Database called department if it doesn't already exist
coll_department = client['department']

department = coll_department.department

In [3]:
#Inserts department names and heads data
department.insert_many([{"dep_name": "IT", "DepartmentHead":"Jason"},
                            {"dep_name": "Admin", "DepartmentHead":"Nial"},
                            {"dep_name": "Accounts", "DepartmentHead":"Harris"}])

##In order to create the data we will pandas dataframe. A Data frame is a two-dimensional data structure, i.e., data is aligned in a tabular fashion in rows and columns. Pandas DataFrame consists of three principal components, the data, rows, and columns.

In [4]:
#This will create a Collection/Database called employees if it doesn't already exist
import pandas as pd

db = client['employees']

df_employees = pd.DataFrame(list(db.employees.find()))

In [5]:
df_employees.head()

,_id,name,Department,Salary
0,60f3d91cee1301e192fab6aa,Jessica,IT,6000
1,60f3d91cee1301e192fab6ab,Joseph,IT,7000
2,60f3d91cee1301e192fab6ac,Alex,Accounts,5000
3,60f3d91cee1301e192fab6ad,Julie,IT,3000
4,60f3d91cee1301e192fab6ae,James,Admin,8000


In [6]:
df_department = pd.DataFrame(list(department.find()))

In [7]:
df_department.head()


,_id,dep_name,DepartmentHead
0,60f3df0b9a2c0576531b768f,IT,Jason
1,60f3df0b9a2c0576531b7690,Admin,Nial
2,60f3df0b9a2c0576531b7691,Accounts,Harris


### Left Join: 

Left join uses only keys from left frame, similar to a SQL left outer join

In [8]:
df_employees.merge(df_department,left_on="Department",right_on="dep_name",how="left")

,_id_x,name,Department,Salary,_id_y,dep_name,DepartmentHead
0,60f3d91cee1301e192fab6aa,Jessica,IT,6000,60f3df0b9a2c0576531b768f,IT,Jason
1,60f3d91cee1301e192fab6ab,Joseph,IT,7000,60f3df0b9a2c0576531b768f,IT,Jason
2,60f3d91cee1301e192fab6ac,Alex,Accounts,5000,60f3df0b9a2c0576531b7691,Accounts,Harris
3,60f3d91cee1301e192fab6ad,Julie,IT,3000,60f3df0b9a2c0576531b768f,IT,Jason
4,60f3d91cee1301e192fab6ae,James,Admin,8000,60f3df0b9a2c0576531b7690,Admin,Nial
5,60f3d91cee1301e192fab6af,Jacob,Admin,9000,60f3df0b9a2c0576531b7690,Admin,Nial
6,60f3d91cee1301e192fab6b0,Kevin,IT,6500,60f3df0b9a2c0576531b768f,IT,Jason


### Inner Join: 

Use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys.

In [10]:
df_employees.merge(df_department,left_on="Department",right_on="dep_name",how="inner")

,_id_x,name,Department,Salary,_id_y,dep_name,DepartmentHead
0,60f3d91cee1301e192fab6aa,Jessica,IT,6000,60f3df0b9a2c0576531b768f,IT,Jason
1,60f3d91cee1301e192fab6ab,Joseph,IT,7000,60f3df0b9a2c0576531b768f,IT,Jason
2,60f3d91cee1301e192fab6ad,Julie,IT,3000,60f3df0b9a2c0576531b768f,IT,Jason
3,60f3d91cee1301e192fab6b0,Kevin,IT,6500,60f3df0b9a2c0576531b768f,IT,Jason
4,60f3d91cee1301e192fab6ac,Alex,Accounts,5000,60f3df0b9a2c0576531b7691,Accounts,Harris
5,60f3d91cee1301e192fab6ae,James,Admin,8000,60f3df0b9a2c0576531b7690,Admin,Nial
6,60f3d91cee1301e192fab6af,Jacob,Admin,9000,60f3df0b9a2c0576531b7690,Admin,Nial


### To get the total number of records in the collection

In [11]:
pipeline = [
     {"$group": {"_id": None,"Count": {"$sum": 1}}}]
grp_employees = db.employees.aggregate(pipeline)


In [12]:
for employee in grp_employees:
  print(employee)

{'_id': None, 'Count': 7}


### To group by department and get the total salary for each department

In [13]:
pipeline = [
     {"$group": {"_id": "$Department","Salary": {"$sum": "$Salary"}}}]
grp_employees = db.employees.aggregate(pipeline)


In [14]:
for employee in grp_employees:
  print(employee)

{'_id': 'IT', 'Salary': 22500}
{'_id': 'Accounts', 'Salary': 5000}
{'_id': 'Admin', 'Salary': 17000}


### To group by department and get the average salary for each department

In [15]:
pipeline = [
     {"$group": {"_id": "$Department","Salary": {"$avg": "$Salary"}}}]
grp_employees = db.employees.aggregate(pipeline)

In [16]:
for employee in grp_employees:
  print(employee['_id'],"\t Average Salary ",employee['Salary'])

IT 	 Average Salary  5625.0
Accounts 	 Average Salary  5000.0
Admin 	 Average Salary  8500.0


### To get the employees with the lowest salary

In [17]:
pipeline = [
     {"$group": {"_id": None ,"Minimum Salary": {"$min": "$Salary"}}}]
grp_employees = db.employees.aggregate(pipeline)

In [18]:
for employee in grp_employees:
    for spec_emp in db.employees.find({"Salary":employee['Minimum Salary']}):
        print("Name ",spec_emp['name'],"\nSalary ",spec_emp['Salary'])




Name  Julie 
Salary  3000
